In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, recall_score, f1_score, roc_auc_score,precision_score
import random
import matplotlib.pyplot as plt

In [2]:
## checando se reconhece a gpu
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
training_percentual = 0.7
matrizes = []
history = []
recall = []
f1score = []
precision = []
images = np.load('imagens.npy')
labels = np.load('labels.npy')

In [4]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, train_size = training_percentual)
X_train, X_val,  y_train, y_val  = train_test_split(X_train, y_train, train_size = 0.8)

In [5]:
X_train.shape

(3697, 300, 300, 3)

In [6]:
X_test.shape

(1982, 300, 300, 3)

In [7]:
X_val.shape

(925, 300, 300, 3)

In [8]:
def buildModel(input_shape=(300, 300, 3)):
    model = Sequential()

    # Camada 1
    model.add(Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((3, 3), strides=(2, 2)))

    # Camada 2
    model.add(Conv2D(256, (5, 5), padding='same', activation='relu'))
    model.add(MaxPooling2D((3, 3), strides=(2, 2)))

    # Camada 3
    model.add(Conv2D(384, (3, 3), padding='same', activation='relu'))

    # Camada 4
    model.add(Conv2D(384, (3, 3), padding='same', activation='relu'))

    # Camada 5
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((3, 3), strides=(2, 2)))

    # Camada fully connected
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

    # output
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model


In [9]:
checkpoint = ModelCheckpoint(
    filepath='C:\\Users\\joao_\\Documents\\Trabalho-ReconhecimentoPadroes\\models\\best_model.hdf5',
    save_weights_only=True,
    monitor='val_loss',
    verbose=1,
    save_best_only=True)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss", 
    factor=0.5, 
    patience=10, 
    min_lr=1e-6, 
    verbose=1)

early_stopping = EarlyStopping(
    monitor="val_loss", 
    patience=30, 
    verbose=1)

In [10]:
model = buildModel()

H = model.fit(X_train, y_train, 
                batch_size=2,
                epochs=100,
                validation_data=(X_val, y_val), 
                verbose=2, 
                callbacks=[checkpoint, reduce_lr, early_stopping])

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
print(y_pred.shape)

y_test = np.argmax(y_test, axis=1)
print(y_test.shape)

#draw_confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred))

In [ ]:
matrizes.append(confusion_matrix(y_test, y_pred))
recall.append(recall_score(y_test, y_pred, average=None))
precision.append(precision_score(y_test, y_pred, average=None))
f1score.append(f1_score(y_test, y_pred, average=None))

In [ ]:
acc = H.history['accuracy']
val_acc = H.history['val_accuracy']
loss = H.history['loss']
val_loss = H.history['val_loss']

plt.figure(figsize=(16,8))
plt.subplot(1, 2, 1)
plt.plot(range(20), acc, label='Acurácia no treino')
plt.plot(range(20), val_acc, label='Acurácia na validação')
plt.legend()
plt.title("Acurácias")

plt.subplot(1, 2, 2)
plt.plot(range(20), loss, label='Loss no treino')
plt.plot(range(20), val_loss, label='Loss na validação')
plt.legend()
plt.title("Loss")

plt.show()